# Exploratory Data Analysis

This notebook runs through a series of exploratory data analyses to assess the integrity and quality of the raw data.

## Installations

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import json
from dataprofiler import Data, Profiler
from functools import reduce

# Set display of images in the notebook
%matplotlib notebook

# imputation libraries
from sklearn.impute import SimpleImputer, KNNImputer

# standardize variables
from sklearn.preprocessing import StandardScaler, MinMaxScaler 

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [6]:
# check versions
print(f'pandas version: {pd.__version__}')
print(f'numpy version: {np.__version__}')
print(f'seaborn version: {sns.__version__}')

pandas version: 1.3.2
numpy version: 1.21.2
seaborn version: 0.12.2


## Basic exploratory data approach

In [19]:
# Read data from csv and set data types (dtype), except the first column -'calldate'- which will be parsed later.
df = pd.read_csv("../data/raw_data.csv", sep=',', engine='python')

In [20]:
# Inspect the dataframe
print(df.shape)
print(df.info())

(10000, 41)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 41 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x0      9993 non-null   float64
 1   x1      9993 non-null   float64
 2   x2      9989 non-null   float64
 3   x3      9986 non-null   float64
 4   x4      9991 non-null   float64
 5   x5      9989 non-null   float64
 6   x6      9992 non-null   float64
 7   x7      9989 non-null   float64
 8   x8      9982 non-null   float64
 9   x9      9985 non-null   float64
 10  x10     9995 non-null   float64
 11  x11     9985 non-null   float64
 12  x12     9992 non-null   float64
 13  x13     9981 non-null   float64
 14  x14     9991 non-null   float64
 15  x15     9992 non-null   float64
 16  x16     9990 non-null   float64
 17  x17     9988 non-null   float64
 18  x18     9991 non-null   float64
 19  x19     9993 non-null   float64
 20  x20     9989 non-null   float64
 21  x21     9995 non-null   

In [21]:
# If necessary, convert columns to str without spaces in lower case
df.columns = df.columns.str.replace('\s+', '_').str.lower()

#### Apply manual functions for data quality checks

In [42]:
### analyze null values
def nullvalues(d):
    ''' Number of null values for each feature and 
        percentage of null values for each feature'''
    print("{:10s}|{:10s}|{:10s}".format("Feature",
                                        "Null values",
                                        "Null Values as a Proportion of Total"))
    print("="*100)
    for col in d.columns:
        null_values = d[col].isnull().sum(axis=0)
        null_values_pct = d[col].isnull().sum(axis=0)/len(d)*100
        print("{:10s}|{:10d}\t|{:10f}".format(str(col),
                                             null_values,
                                             null_values_pct))

### analyze cardinality
def cardinality(data):
    ''' Check number of unique values of variables
        not accounting for null values '''
    print("{:15s}\t| {:10s}\t| {:10s}".format("Feature",
                                              "Distinct Values",
                                              "Distinct Values as a Proportion of Total"))
    print("="*100)
    for col in data.columns[:]:
        unique_values = len(np.unique(data[col].ffill()))
        unique_values_pct = len(np.unique(data[col].ffill()))/len(data) 
        print("{:15s}\t| {:10d}\t\t| {:10f}".format(str(col),
                                                   unique_values,
                                                   unique_values_pct))

### analyze duplicates
def rowduplication(data):
    ''' Assess the number and percent of duplicates for entire rows in dataset '''
    data_dedup = data.drop_duplicates(keep='first')
    data_duplicates = data[data.duplicated(subset=None, keep='first')]
    
    print('Shape of de-duplicated dataset', data_dedup.shape)
    print('Number of duplicates:', len(data) - len(data_dedup))
    print('Confirm number of duplicates:', len(data_duplicates))

def duplicationanalyzer(data):
    ''' Assess the number and percentage of duplicates 
        for each variable in the dataset'''
    variable = pd.Series(np.nan)
    
    for var in data:
        ''' Output the number of duplicates and proportion of duplicates '''
        variable = data[var]
        
        # construct variable dataframe less duplicates
        variable_dedup = variable.drop_duplicates(keep='first')
        
        #print the number of duplicates
        duplicates=data[data[var].duplicated(keep=False)]
        print(var)
        print('Number of duplicates: ', len(duplicates))
        
        #print the percentage of duplicates
        percentage = "{0:.4f}".format(len(duplicates)/len(data))
        print('Percentage of duplicates: ', percentage)

In [40]:
nullvalues(d=df)

Feature   |Null values|Null Values as a Proportion of Total
x0        |         7	|  0.070000
x1        |         7	|  0.070000
x2        |        11	|  0.110000
x3        |        14	|  0.140000
x4        |         9	|  0.090000
x5        |        11	|  0.110000
x6        |         8	|  0.080000
x7        |        11	|  0.110000
x8        |        18	|  0.180000
x9        |        15	|  0.150000
x10       |         5	|  0.050000
x11       |        15	|  0.150000
x12       |         8	|  0.080000
x13       |        19	|  0.190000
x14       |         9	|  0.090000
x15       |         8	|  0.080000
x16       |        10	|  0.100000
x17       |        12	|  0.120000
x18       |         9	|  0.090000
x19       |         7	|  0.070000
x20       |        11	|  0.110000
x21       |         5	|  0.050000
x22       |        12	|  0.120000
x23       |         8	|  0.080000
x24       |        10	|  0.100000
x25       |        10	|  0.100000
x26       |        12	|  0.120000
x27       |        13	

In [36]:
cardinality(data=df)

Feature        	| Distinct Values	| Distinct Values as a Percent of Total
x0             	|       9993		|   0.999300
x1             	|       9993		|   0.999300
x2             	|       9989		|   0.998900
x3             	|       9986		|   0.998600
x4             	|       9991		|   0.999100
x5             	|       9989		|   0.998900
x6             	|       9992		|   0.999200
x7             	|       9989		|   0.998900
x8             	|       9982		|   0.998200
x9             	|       9985		|   0.998500
x10            	|       9995		|   0.999500
x11            	|       9985		|   0.998500
x12            	|       9992		|   0.999200
x13            	|       9981		|   0.998100
x14            	|       9991		|   0.999100
x15            	|       9992		|   0.999200
x16            	|       9990		|   0.999000
x17            	|       9988		|   0.998800
x18            	|       9991		|   0.999100
x19            	|       9993		|   0.999300
x20            	|       9989		|   0.998900
x21            	|      

In [37]:
rowduplication(data=df)

Shape of de-duplicated dataset (10000, 41)
Number of duplicates: 0
Confirm number of duplicates: 0


In [43]:
duplicationanalyzer(data=df)

x0
Number of duplicates:  7
Percentage of duplicates:  0.0007
x1
Number of duplicates:  7
Percentage of duplicates:  0.0007
x2
Number of duplicates:  11
Percentage of duplicates:  0.0011
x3
Number of duplicates:  14
Percentage of duplicates:  0.0014
x4
Number of duplicates:  9
Percentage of duplicates:  0.0009
x5
Number of duplicates:  11
Percentage of duplicates:  0.0011
x6
Number of duplicates:  8
Percentage of duplicates:  0.0008
x7
Number of duplicates:  11
Percentage of duplicates:  0.0011
x8
Number of duplicates:  18
Percentage of duplicates:  0.0018
x9
Number of duplicates:  15
Percentage of duplicates:  0.0015
x10
Number of duplicates:  5
Percentage of duplicates:  0.0005
x11
Number of duplicates:  15
Percentage of duplicates:  0.0015
x12
Number of duplicates:  8
Percentage of duplicates:  0.0008
x13
Number of duplicates:  19
Percentage of duplicates:  0.0019
x14
Number of duplicates:  9
Percentage of duplicates:  0.0009
x15
Number of duplicates:  8
Percentage of duplicates:  0

### DataProfiler approach

Use the [dataprofiler library](https://pypi.org/project/DataProfiler/) to assess the data.

In [12]:
data = Data("../data/raw_data.csv")

In [13]:
print(data.data.head(5)) # Access data directly via a compatible Pandas DataFrame

                     x0                    x1                   x2  \
0    0.3026413389641203    -1.214299389169045  -1.2378507017550309   
1    0.8642862878515198  -0.38437037248885153   0.7327842003709444   
2  -0.08155094236032266    0.2769807484072984  0.44719830109263947   
3   -1.8390669001683608   -0.2304594155134607   0.7073266410818246   
4  -0.21999078219905976    0.9189226165187019   1.3462777577844436   

                     x3                    x4                    x5  \
0     1.500563662188802   -0.8092440903844441   -0.4530452113971498   
1   -1.5366619151095287  -0.04054563464470142    1.3587527903813474   
2  0.004848440880356852   -1.3271490918573094   0.45272005625953293   
3    0.3679564917014473   -0.6125171856217925  -0.32625401311139324   
4    1.1564184430868125   -0.6685773219031153    0.9144162168556297   

                    x6                   x7                    x8  \
0  -0.6819770871057071   2.4345651949244393   -1.0053044050317834   
1  -1.50518275

In [14]:
profile = Profiler(data) # Calculate Statistics, Entity Recognition, etc

INFO:DataProfiler.profilers.profile_builder: Finding the Null values in the columns...  (with 9 processes)
INFO:DataProfiler.profilers.profile_builder: Processing Column 1/41
INFO:DataProfiler.profilers.profile_builder: Processing Column 2/41
INFO:DataProfiler.profilers.profile_builder: Processing Column 3/41
INFO:DataProfiler.profilers.profile_builder: Processing Column 4/41
INFO:DataProfiler.profilers.profile_builder: Processing Column 5/41
INFO:DataProfiler.profilers.profile_builder: Processing Column 6/41
INFO:DataProfiler.profilers.profile_builder: Processing Column 7/41
INFO:DataProfiler.profilers.profile_builder: Processing Column 8/41
INFO:DataProfiler.profilers.profile_builder: Processing Column 9/41
INFO:DataProfiler.profilers.profile_builder: Processing Column 10/41
INFO:DataProfiler.profilers.profile_builder: Processing Column 11/41
INFO:DataProfiler.profilers.profile_builder: Processing Column 12/41
INFO:DataProfiler.profilers.profile_builder: Processing Column 13/41
INFO:

Create a readable report

In [15]:
readable_report = profile.report(report_options={"output_format": "compact"})

In [16]:
readable_report

{'global_stats': {'samples_used': 5000,
  'column_count': 41,
  'row_count': 10000,
  'row_has_null_ratio': 0.041,
  'row_is_null_ratio': 0.0,
  'unique_row_ratio': 1.0,
  'duplicate_row_count': 0,
  'file_type': 'csv',
  'encoding': 'utf-8',
  'correlation_matrix': None,
  'chi2_matrix': '[[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,\n  nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,\n  nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], ... , [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,\n  nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,\n  nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,  1.]]',
  'profile_schema': {'x0': [0],
   'x1': [1],
   'x2': [2],
   'x3': [3],
   'x4': [4],
   'x5': [5],
   'x6': [6],
   'x7': [7],
   'x8': [8],
   'x9': [9],
   'x10': [10],
   'x11': [11],
   'x12': [12],
   'x13': [13],
   'x14': [14],
   'x15': [15],
   'x16': [16],
   '